In [5]:
pip install jsonlines


Note: you may need to restart the kernel to use updated packages.


In [1]:
import jsonlines
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import evaluate
import datasets
from transformers import Trainer, TrainingArguments
from transformers import CamembertTokenizer, CamembertForSequenceClassification

jsonl_file_path = 'data/french_books_reviews.jsonl'

l = []
with jsonlines.open(jsonl_file_path) as reader:
    for line in reader:
        l.append(line)


df = pd.DataFrame(l)

/Users/hervetrinh/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-07 16:47:57.777298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df

,book_title,author,reader_review,rating,label
0,Le Démon de la Colline aux Loups,Dimitri Rouchon-Borie,Ce n'est pas le premier roman à aborder les th...,5.0,1
1,Simple,Marie-Aude Murail,"Simple, alias Barnabé, est un jeune homme de 2...",4.0,1
2,La plus secrète mémoire des hommes,Mohamed Mbougar Sarr,Pour écrire La plus secrète mémoire des hommes...,4.0,1
3,Trancher,Amélie Cordonnier,« La violence d'Aurélien est revenue. Par la f...,3.5,0
4,"La guerre d'Alan, tome 2",Emmanuel Guibert,"Dans ce second album de La Guerre d’Alan, Emma...",5.0,1
...,...,...,...,...,...
9653,Un grand méchant loup !,Geoffroy de Pennart,Geoffroy de Pennart est un auteur-illustrateur...,4.0,1
9654,Travailler: La grande affaire de l'humanité,James Suzman,Seule l’énergie permet au vivant de résister à...,4.0,1
9655,Leur domaine,Jo Nesbø,Sauf que ce n’est pas publié par Gallmeister… ...,4.0,1
9656,Ivan le terrible,Anne Fine,Un jeune Russe arrive dans l’école du narrateu...,4.0,1


In [3]:
df.label.value_counts()

label
 1    6670
 0    2129
-1     859
Name: count, dtype: int64

In [4]:
df[df.label == -1]

,book_title,author,reader_review,rating,label
32,Sable bleu,Yves Grevet,"C'est vrai, je ne suis pas le public ciblé, pu...",2.0,-1
88,Jeux de ficelle,Nina Vincent,Les enfants aiment toujours autant jouer avec ...,2.5,-1
171,Vous faites quoi pour Noël ?,Carène Ponte,"Avec un titre pareil, on se doute que ce roman...",2.0,-1
209,La Vérité sur la lumière,ÓLAFSDÓTTIR AUDUR AVA,Le mois de décembre n’a pas été un bon cru pou...,2.5,-1
240,"Un c'est bien, deux c'est mieux, tome 1",Dana Blue,"Voici que j'achève, avec ce roman, les cinq ou...",1.0,-1
...,...,...,...,...,...
9627,La définition du bonheur,Catherine Cusset,Contrairement à ce que pourrait laisser entend...,2.5,-1
9628,"Reine Kannon Ka, la vérité interdite",Apollo Blueskin,"Il y a huit mille ans, un gigantesque vaisseau...",1.5,-1
9636,L'Unique goutte de sang,Arnaud Rozan,"Peut-on radiographier la haine, disséquer les ...",2.5,-1
9650,La France n'a pas dit son dernier mot,Éric Zemmour,Après le retentissant succès du « Suicide fran...,2.5,-1


In [5]:
label_encoder = LabelEncoder()

df.label = label_encoder.fit_transform(df.label)

/Users/hervetrinh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/hervetrinh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [20]:
df

,text,label
0,Ce n'est pas le premier roman à aborder les th...,2
1,"Simple, alias Barnabé, est un jeune homme de 2...",2
2,Pour écrire La plus secrète mémoire des hommes...,2
3,« La violence d'Aurélien est revenue. Par la f...,1
4,"Dans ce second album de La Guerre d’Alan, Emma...",2
...,...,...
9653,Geoffroy de Pennart est un auteur-illustrateur...,2
9654,Seule l’énergie permet au vivant de résister à...,2
9655,Sauf que ce n’est pas publié par Gallmeister… ...,2
9656,Un jeune Russe arrive dans l’école du narrateu...,2


In [6]:
df = df[["reader_review", "label"]]

df.columns = ["text", "label"]

In [8]:
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=3)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
ds = datasets.Dataset.from_pandas(df)

In [11]:
ds

Dataset({
    features: ['text', 'label'],
    num_rows: 9658
})

In [12]:
ds = ds.train_test_split(test_size=0.2)

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/7726 [00:00<?, ? examples/s]

Map: 100%|██████████| 1932/1932 [00:00<00:00, 2487.07 examples/s]


In [14]:
ds_train = tokenized_datasets["train"]
ds_test = tokenized_datasets["test"]

In [15]:
training_args = TrainingArguments(output_dir="output/", 
                                  num_train_epochs=5,
                                  logging_dir="logs/",
                                  learning_rate=1.e5)

In [16]:
metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

  0%|          | 17/4830 [14:38<69:04:54, 51.67s/it]
